# Store Sales - Time Series Forecasting
Using time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer.

The goal of the project is to build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.

The fields of the data are:
* store_nbr identifies the store at which the products are sold.
* family identifies the type of product sold.
* sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
* onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.
* Store metadata, including city, state, type, and cluster. 
    * cluster is a grouping of similar stores.
* Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

### Additional Notes
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

C:\Users\hp\AppData\Local\Temp\ipykernel_11060\3790897354.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Importing Data and Combining dataframes

In [2]:
df = pd.read_csv('train.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')
holidays = pd.read_csv('holidays_events.csv')

In [3]:
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [5]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [6]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [8]:
# Merging the different dataframes
df = pd.merge(df, stores, how='left', on='store_nbr')
df = pd.merge(df, oil, how='left', on='date')
df = pd.merge(df, holidays, how='left', on='date')
df = pd.merge(df, transactions, how='left', on=['date', 'store_nbr'])

In [9]:
df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN


## Data Cleaning

In [10]:
df.drop('id', axis=1, inplace=True)

In [11]:
df.describe()

,store_nbr,sales,onpromotion,cluster,dcoilwtico,transactions
count,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,2.099196e+06,2.805231e+06
mean,2.750000e+01,3.590209e+02,2.617480e+00,8.481481e+00,6.801587e+01,1.697071e+03
std,1.558579e+01,1.107286e+03,1.225494e+01,4.649735e+00,2.569134e+01,9.668317e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.619000e+01,5.000000e+00
25%,1.400000e+01,0.000000e+00,0.000000e+00,4.000000e+00,4.641000e+01,1.046000e+03
50%,2.750000e+01,1.100000e+01,0.000000e+00,8.500000e+00,5.343000e+01,1.395000e+03
75%,4.100000e+01,1.960110e+02,0.000000e+00,1.300000e+01,9.581000e+01,2.081000e+03
max,5.400000e+01,1.247170e+05,7.410000e+02,1.700000e+01,1.106200e+02,8.359000e+03


In [12]:
df.isnull().sum()

date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
dcoilwtico       955152
type_y          2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
transactions     249117
dtype: int64

In [13]:
df['dcoilwtico'] = df['dcoilwtico'].fillna(method='backfill')

C:\Users\hp\AppData\Local\Temp\ipykernel_11060\2024084909.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['dcoilwtico'] = df['dcoilwtico'].fillna(method='backfill')


In [14]:
df.isnull().sum()

date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
dcoilwtico            0
type_y          2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
transactions     249117
dtype: int64

In [15]:
df['transactions'].replace(np.nan, 0, inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_11060\1782814881.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transactions'].replace(np.nan, 0, inplace=True)


In [16]:
df.head()

,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.0
1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.0
2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.0
3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.0
4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.0


## Feature Engineering

* Create new features from existing columns
* Label encode data
* Normalize

In [17]:
# splitting the date into year, month and day
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [18]:
# Creating new feature that shows if its payday or not
df['pay_day'] = df['date'].apply(lambda x: 1 if x.day == 15 or x.is_month_end else 0)

In [19]:
data = df.drop('date', axis=1)
data.drop('description', axis=1, inplace=True)

In [20]:
data.head()

,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,transferred,transactions,year,month,day,pay_day
0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,False,0.0,2013,1,1,0
1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,False,0.0,2013,1,1,0
2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,False,0.0,2013,1,1,0
3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,False,0.0,2013,1,1,0
4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,False,0.0,2013,1,1,0


In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

columns = ['family', 'city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'transferred']
for col in columns:
    data[col] = le.fit_transform(data[col])

data.head()

,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,transferred,transactions,year,month,day,pay_day
0,1,0,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
1,1,1,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
2,1,2,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
3,1,3,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
4,1,4,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0


In [22]:
data.head()

,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,transferred,transactions,year,month,day,pay_day
0,1,0,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
1,1,1,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
2,1,2,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
3,1,3,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0
4,1,4,0.0,0,18,12,3,13,93.14,3,1,4,0,0.0,2013,1,1,0


# Model Building

In [23]:
# Splitting the data into labels and target

X = data.drop('sales', axis=1)
y = data['sales']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2443478, 17)
(610870, 17)
(2443478,)
(610870,)


In [24]:
# Scaling the data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [25]:
X_train_sc

array([[ 1.69995522, -1.57524947, -0.21369398, ..., -1.54803108,
        -0.18832439, -0.26341708],
       [ 1.05840767, -0.3149924 , -0.21369398, ..., -0.36251064,
        -1.43812783, -0.26341708],
       [ 0.60932438,  1.68041462, -0.21369398, ..., -0.65889075,
        -0.75641686, -0.26341708],
       ...,
       [-0.35299695,  0.94526466, -0.21369398, ..., -1.54803108,
         0.0389126 , -0.26341708],
       [-1.70024681, -0.84009951,  1.33507211, ..., -1.25165097,
        -0.18832439, -0.26341708],
       [-0.41715171, -0.20997098, -0.21369398, ..., -0.65889075,
         1.06147906, -0.26341708]])

In [26]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train_sc, y_train )
reg.score(X_test_sc, y_test)

0.2288532377631376

In [27]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train_sc, y_train)
xgb_reg.score(X_test_sc, y_test)


0.9009500992507559

In [28]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=0)
tree.fit(X_train_sc, y_train)
tree.score(X_test_sc, y_test)

0.8939335617912544

# Model Evaluation